# Рынок BMW

## Описание

Для простоты рассмотрим рынок BMW, выпущенных после 2010 года. Данные берутся с 3 наиболее популярных сайтов по продаже авто в России — Авто.ру, Авито, Дром

## Сбор данных

Так как каждый сайт имеет свой HTML-код, то код для парсинга каждого сайта будет отличаться. Используется динамический парсинг с помощью Selenium

### Основные критерии

- Модель
- Год выпуска
- Объем двигателя
- Бензин/дизель
- Привод
- Мощность (л.с.)
- Пробег
- КПП
- Цена
- Тип (седан/внедорожник/кроссовер) (???)


In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import time
import auto_ru_parse

### Парсинг с Авто.ру

In [ ]:
class Auto:
    def __init__(self, model: str = "", gen: str = "", year: int = 0, engine_volume: float = 0.0, fuel_type: str = "",
                 drive: str = "", power_hp: int = 0, mileage: int = 0, transmission: str = "", price: int = 0, url: str = ""):
        self.model = model                      # модель
        self.gen = gen                          # серия
        self.year = year                        # год выпуска
        self.engine_volume = engine_volume      # объем двигателя
        self.fuel_type = fuel_type              # бензин/дизель
        self.drive = drive                      # привод
        self.power_hp = power_hp                # мощность в л.с.
        self.mileage = mileage                  # пробег (==0, если новая)
        self.transmission = transmission        # КПП
        self.price = price                      # цена
        self.url = url                          # ссылка на сайт

    def __str__(self):
        return (f"{self.model} | {self.gen} | {self.year} | {self.mileage} |{self.engine_volume}L | {self.power_hp} HP | {self.fuel_type} | "
                f"{self.transmission} | {self.drive} | {self.price} ₽ | {self.url}")


In [ ]:
num_pages = 10                      # кол-во страниц, которые хоти обработать
year = 2010                         # самый ранний год выпуска машины
non_dymanic = True                  # если нужно, чтобы браузер НЕ открывался в явном виде, то поставить True

In [ ]:
htmls = auto_ru_parse.parse_html(year, num_pages, non_dymanic)

In [ ]:
cars = auto_ru_parse.collect_data(htmls)

In [ ]:
for el in cars:
    print(el)